Please type your name in the cell below: >>>

# Nitrogen Concentration and Nitrogen Deposition practical

### Notebook by: Antonija Rimac and Michiel van der Molen


## Contents
1. Introduction 

1. Atmospheric NH$_3$ concentrations  
    2.1 Data Exploration  
    2.2 Variability in NH3 concentrations (inter-annual, seasonal, and diurnal)     

1. Dry deposition of NH$_3$  
    3.1 Aerodynamical resistance  
    3.2 Canopy resistance    
   
1. SAVE YOUR RESULTS, CLEAR OUTPUT and CLOSE NOTEBOOK  


## 1 Introduction

The Basic Skills - Air Quality focuses on atmospheric nitrogen concentration and nitrogen deposition. "Nitrogen" here refers to reactive nitrogen (N$_r$) in the form of ammonia (NH$_3$). The atmospheric NH$_3$ concentration depends on emission rates and atmospheric dispersion, as a function of boundary layer mixing. Deposition of NH$_3$ co-determines nutrient variability in agricultural and natural ecosystems. Nutrient availability affects vegetation growth rates and water quality. Figure 1 shows the modelled deposition of N$_r$ over land in the Netherlands in 2018 and the positions of three stations for which we will calculate the nitrogen deposition in this practical, and the position of Wageningen Veenkampen station, where the meteorological station is situated.

<br>
<div align=center>
<figure>
  <img src="Fig_01_NitrogenDepositionMap.png", width="700" height="600">
  <figcaption> <i>Figure 1: Spatial distribution of the calculated deposition of reactive nitrogen in the Netherlands in 2018. </i></figcaption>
</figure>
</div>

Section 2 (Atmospheric NH$_3$ concentrations) is based on the MAQ10306 Introduction Atmosphere course (see chapter 1 "The Earth and its atmosphere", and chapter 8 "The atmospheric boundary layer"), while section 3 (Dry deposition of NH$_3$) follows up on chapter 18 ("Exchange of nitrogen species at the Earth's surface").

Today you will explore the variability of nitrogen concentrations. You will develop a model to simulate the dry deposition rate of NH$_3$ on vegetated surfaces. **You will need this model later on to successfully execute the Integration Skills and to work in the PBL part.** 

The objectives of the following exercises are:

* To explain variations in atmospheric NH$_3$ concentrations (Section 2).
* To construct a model to simulate dry deposition of NH$_3$ (Section 3).
* To explain variations in NH$_3$ dry deposition rates (Section 3).
* To use the model for sensitivity studies and uncertainty analyses (Section 4).

You will use all the results you obtain today later in the course. Therefore, do not forget to write your answers at the intended locations and save your results. Also, if necessary, make notes about the things you learn.  

<p style="background:powderblue;color:black">(Python coding explanation:) > Some parts of the notebook are marked in (powder)blue. These parts give some more in-depth explanation about coding in Python/Jupyter Notebooks.<br></p>

Note that the "Jupyter notebook" practical consists of so-called code cells and markdowns cells. Markdown cells are used to write text, pose questions, and for you to write your notes. Code cells are used to write parts of a numerical code or program. In each code cell we write a necessary part of a code and we write some comments that start with the mark "#". In the comment, we explain (again) what we calculate or plot. To obtain a result of a calculation i.e., a figure (note that most of the calculations we have already done for you), you will need to run a cell. You can do that by clicking on a cell and pressing simultaneously **Shift Enter**. Only then you will be able to see the result of the calculation and only then will the figures appear on your screen. Happy notebooking!

___
## Setting up Python modules

We start the tutorial by loading the required Python packages. In the cell below we load the necessary python packages (if some warning signs or requirement signs appear, ignore them).

In [ ]:
# Load necessary python packages. This is needed to give Python the proper functionality. 
#import sys
#!{sys.executable} -m pip install cufflinks > /dev/null; # Remove > /dev/null in case of errors.
import os
teacher_dir  = os.getenv('TEACHER_DIR')

import sys
sys.path.append(teacher_dir + '/JHL_data/pictures/')

from   ipywidgets import interact
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from   pandas.tseries.frequencies import to_offset
import cufflinks as cf
import plotly.express as px
%matplotlib inline
import spaceship as ss

# (Ignore possible warnings, i.e., pretend they did not show up)

## 2 Atmospheric NH$_3$ concentrations

In this section, you will analyze the atmospheric NH$_3$ concentrations provided by the National Monitoring network Air Quality (Landelijk Meetnet Luchtkwaliteit, LML, <a href="https://www.luchtmeetnet.nl/meetpunten?component=NH3">luchtmeetnet.nl</a>). In the cell below, we load the NH$_3$ concentrations as hourly values at the Wekerom, Vredepeel and Zegveld stations.

<p style="background:powderblue;color:black">
You could take a minute to inspect the data. You can do so by going to the Jupyter file manager tab. Then navigate to the directory JHL_data/IC_data and check which files are there. The files are .csv files, which means they are simple text files, which contain hourly data. The column title tells the notebook which column to import. The files can also be easily imported to MS Excel. Open one or two files to check out the file structure. Note that you can easily add columns or manipulate the data.</p>

In [ ]:
# Load the values of nitrogen NH3 concentration for three different stations 
# (Wekerom, Vredepeel and Zegveld). 
# Data are loaded as hourly values in ug/m3.
datapath     = teacher_dir + '/JHL_data/IC_data/'

df_Wekerom   = pd.read_csv(datapath + 'Data_AQ_Wekerom_2013-2021.csv', sep=';', 
    index_col='date-time', usecols=['date-time', 'NH3'], parse_dates=['date-time'])
df_Wekerom.columns = ['NH3_Wekerom']

df_Vredepeel = pd.read_csv(datapath + 'Data_AQ_Vredepeel_2013-2021.csv', sep=';', 
    index_col='date-time', usecols=['date-time', 'NH3'], parse_dates=['date-time'])
df_Vredepeel.columns = ['NH3_Vredepeel']

df_Zegveld   = pd.read_csv(datapath + 'Data_AQ_Zegveld_2013-2021.csv', sep=';', 
    index_col='date-time', usecols=['date-time', 'NH3'], parse_dates=['date-time'])
df_Zegveld.columns = ['NH3_Zegveld']

# Save the data from Wekerom, Vredepeel and Zegveld in the same dataframe.
df_airquality = pd.concat([df_Wekerom, df_Vredepeel, df_Zegveld], axis=1, sort=False)

### Exercise 1: Data exploration

In the cell above we loaded the ammonia concentration data. Now let's look at the data for the three stations. 

In this exercise, you will analyze the variability of NH$_3$ concentrations at three different stations (i.e., at Wekerom, Vredepeel and Zegveld). We uploaded the data for you and we wrote the code that will make initial figures (note: sometimes this may take a while to execute). Below the figures, you will find some questions you should answer by analyzing the figures. You will write your answers as a piece of code. We will guide you along the way. When you are done with your calculation and when it is successful, you can save your answers by going to *File > Save as*. Note that you have to execute each cell below in the order of appearance.

The hourly NH$_3$ concentrations show a lot of variability over 8 years and 3 stations. Let's look at the differences in the mean value.

### Question 1.1:  

1. Which station has the highest mean ammonia concentration?<br>
2. How high is this mean concentration?

To answer question 1.1 do the following:

<ul>
<li><p style="background:powderblue;color:black">Calculate the mean value for the three stations as: df_mean_stationname = df_stationname.mean(). Here .mean() means that we calculate the mean value of all data points in our time series. Note that instead of _stationname you should write _Wekerom, _Vredepeel or _Zegveld.</li></ul>

- Write your results in the second cell box below. To answer the *first* question, you should write: `x = 'stationname'`. To answer the *second* question, you should write: `y = number`.  

In [ ]:
# Calculate the mean value for the different stations.
df_mean_Wekerom   = df_Wekerom.mean()
df_mean_Vredepeel = df_Vredepeel.mean()
df_mean_Zegveld   = df_Zegveld.mean() 

# Print the result of the calculated mean value on your screen. 
# The mean value is given in ug/m3.
NH3_units = '\u00b5g m\u207B\u00b3' # ug/m3
print('Mean concentration at station %10s is %7.2f %s'%('Wekerom',   df_mean_Wekerom,   NH3_units))
print('Mean concentration at station %10s is %7.2f %s'%('Vredepeel', df_mean_Vredepeel, NH3_units))
print('Mean concentration at station %10s is %7.2f %s'%('Zegveld',   df_mean_Zegveld,   NH3_units))

In [ ]:
# Write your answers:
# x: Which station has the highest mean ammonia concentration?; x = 'Stationname'
# y: What is this mean concentration?; y = number (2 decimals)
x = 'Vredepeel'     
y = 17          

ss.check_my_answer_AQ1_1(x, y)

### Question 1.2:

Name 3 reasons for the variability seen in the table above (think about what you have learned in the Introduction Atmosphere). 

Possible answers are: <br>
a *Wind direction*, <br>
b *Wind speed*, <br>
c *Changes in emissions*, <br>
d *Something else*.

To answer question 1.2 do the following:

- Write your results in the cell box below. You can do that as: `x = 'letter'`. If there is more than one correct answer, separate each answer by a comma, and write each letter separately under `''` (e.g., x = 'a','d'). 

In [ ]:
# Name 3 reasons for the variability seen in the table above (think about what you have learned
# in Introduction Atmosphere). 
# Multiple answers may be true, e.g.: x = 'a' or x = 'b','c'.
x = 'a','b','c'           

ss.check_my_answer_AQ1_2(x)

### Exercise 2: Variability in NH$_3$ concentrations

In this exercise, you will analyze the variability in NH$_3$ concentrations. You will look at the variability on a. inter-annual, and b. seasonal scales. You will compare the results obtained at the three stations. Here, you will again perform some calculations.

### Exercise 2a: Inter-annual variability

You will first look at the inter-annual variability.

### Question 2a.1:  

1. Which station has the highest inter-annual mean value? <br>
2. How large is the inter-annual variability in % at that station? <br>
3. What can cause such big differences in the inter-annual variability between the three stations?

To answer question 2a.1 do the following:

<ul>
<li><p style="background:powderblue;color:black">Calculate the inter-annual variability for the three stations as: "df_mean_interannual = df_airquality.resample('Y').mean()". Note that "resample('Y')" means that we resample our data so that we could perform calculations for each year separately. </li><br>
    
<li><p style="background:powderblue;color:black">Plot the inter-annual variability for the three stations.</li></ul>

- Write your results in the second cell box below. To answer the *first two* questions, you should write: `x = 'stationname'` and `y = integer number`, respectively. You can use equation 100%*(max-min)/mean (+/10%) to make a calculation to answer the *second* question. To answer the *third* question, you write: `z = letter`. Here `letter` corresponds to one of the following explanations: <br>
a *The highest inter-annual mean value happens because the station is located near the coast where easterly winds introduce polluted air, consequently increasing NH$_3$ concentrations.*<br>
b *The highest inter-annual mean value happens because the station is located in the Brabant area where (pig)farms emit a lot of nitorgen.*<br>
c *The lowest inter-annual mean value happens because the station is located near the coast where westerly winds introduce clean air, consequently reducing NH$_3$ concentrations.*  
Note that more than one answers are correct.

In [ ]:
# Calculate the inter-annual variability for the three stations.
# Inter-annual variability is calculated as a mean value for each year of our time-series.
df_mean_interannual = df_airquality.resample('Y').mean()

# remove the month and day from the index, they are not relevant any longer.
df_mean_interannual.index = df_mean_interannual.index.strftime("%Y")

# Plot the result (inter-annual variability) for the three stations 
# (Wekerom in orange, Vredepeel in blue and Zegveld in green). 
# Plot Time [years] on x-axis and Concentration [um/m^3] on y-axis.  
fig2 = df_mean_interannual.iplot(asFigure=True, xTitle="Time [years]", \
                                 yTitle="NH\u2083 Concentration  [" + NH3_units +"]", width=2)
fig2.show()

#Note that you can zoom in the specific parts of the figure by using your cursor or the controls on top of the figure.

In [ ]:
# Write your answers:
# x: Which station has the highest inter-annual mean value? x = 'stationname'
# y: How large is the inter-annual variability in % at that station? y = integer; 100%*(max-min)/mean (+/10%)?
# z: What can cause such big differences in the inter-annual variability between the stations? 
# Multiple answers may be true, e.g.: z = 'a' or z = 'b','c'.
x = 'Vredepeel'          
y = 67           
z = 'b','c'          
              
ss.check_my_answer_AQ2a_1(x, y, z)

### Exercise 2b: The seasonal cycle

Now look at the multi-year seasonal cycle in NH$_3$ concentrations.

### Question 2b.1: 

1. Which station has the largest amplitude in the multi-year seasonal variability in NH$_3$ concentration? <br>
2. How many dominant peaks can you identify in the multi-year seasonal variability in NH$_3$ concentration for all three stations? <br> 
3. At which weeks of the year do you identify these peaks? <br>

To answer question 2b.1 do the following:

<ul>
<li><p style="background:powderblue;color:black"> Calculate the multi-year seasonal variability for the three stations. The seasonal variability can then be calculated as: "df_mean_seasonal = df_airquality.groupby(2*((df_airquality.index.week-1)//2 + 1)).mean()". Here, we calculate the multi-year seasonal variability by first indexing all the weeks in a year, and then we calculate the mean value of all weeks with the same index in the time series. </li><br>
    
<li><p style="background:powderblue;color:black">Plot the multi-year seasonal variability for all three stations. In this figure, we plot the Time [weeks] on x-axis and the NH$_3$ Concentration [μg/m$^3$] on y-axis. </li></ul>
 
- Analize the figure and answer the posed questions in the second cell box below. To answer the *first* question, you  write: `x = 'stationname'`. To answer the *second two* questions, you will write `y = number`, `z1 = number`, and `z2 = number`, where `z1` and `z2` correspond to the first and second identified peek, respectively. 
   

In [ ]:
# Calculate the mean seasonal variability for all three stations. 
# The mean seasonal variability can be calculated as a mean of all 
# e.g., first weeks of the year. 
df_mean_seasonal = df_airquality.groupby(2*((df_airquality.index.isocalendar().week-1)//2 + 1)).mean()

# Plot the result (mean seasonal cycle) for all three stations 
# (Wekerom in orange, Vredepeel in blue and Zegveld in green). 
# Plot Time [weeks] on x-axis and Concentration [um/m^3] on y-axis.
fig3 = df_mean_seasonal.iplot(asFigure=True, xTitle="Time [weeks]", \
                              yTitle="NH\u2083 Concentration [" + NH3_units + "]", width=2)
fig3.show() 

In [ ]:
# Write your answers:
# x: Which station has the largest amplitude in the mean seasonal variability in NH3 concentration? 
# x = 'Stationname'
# y: How many dominant peaks can you identify in the mean seasonal variability in NH3 concentration 
# for the three stations? y = integer
# z: At which weeks of the year do you identify these peaks? z1 = integer, z2 = integer, z3 = integer
x  = 'Vredepeel'         
y  = 3         
z1 = 14         
z2 = 30
z3 = 48

ss.check_my_answer_AQ2b_1(x, y, z1, z2, z3)

Additional questions: How does applying the manure on the agricultural fields influence the identified peaks?

Note that the seasonal differences largely depend on manure spreading. Check the manure season at https://www.rvo.nl/onderwerpen/agrarisch-ondernemen/mestbeleid/gebruiken-en-uitrijden/wanneer-mest-uitrijden and compare to the NH$_3$ concentrations given in the figure above.

---
## 3 Dry Deposition of NH$_3$ 

Ammonia is very soluble in water. Therefore, dry deposition of ammonia is most effective on wet surfaces. Deposition on dry leafs or soil is orders of magnitude slower, and thus it will be neglected in this course. Vegetation is obviously wet after precipitation and dew, but this happens relatively infrequently. A major location of ammonia deposition is in the stomates, since they are wet (the plants regulate the stomatal opening to prevent excessive water loss from the stomates).

Here, we will describe the dry deposition rate $F_{NH_3}$ using a gradient-resistance model:

\begin{equation*}
F_{NH_3} = \frac{C_{NH_3,\alpha}-C_{NH_{3,0}}}{r_t} .
\end{equation*}

In this equation, $C_{NH_3}$ (μg m$^{-3}$) is the atmospheric concentration of NH$_3$, while $r_t$ (s m$^{-1}$) is the total resistance. The subscripts $\alpha$ and $0$ refer to the atmospheric reference level and the zero-level (where the NH$_3$ is absorbed). The atmospheric concentrations were already discussed in section 2.

In this Ohm law-like expression, the total resistance $r_t$ is composed of the aerodynamic resistance $r_a$, the sub-laminar boundary layer resistance $r_b$ and the canopy resistance $r_c$ (see Figure 2). In this form, the three resistances are in series:

\begin{equation*}
r_t = r_a + r_b + r_c .
\end{equation*}

In this section, we will explain and discuss how the resistances and the deposition rate are calculated.

<br>
<div align=center>
<figure>
  <img src="Fig_02_Resistances.png", width="700" height="600">
  <figcaption> <i>Figure 2: Three different resistances and the level of their influence. </i></figcaption>
</figure>
</div>


Note that the process of ammonia deposition is far more complicated than represented here. In practice, ammonia deposits on wet surfaces, from which the water may evaporate along with the ammonia. Plants may emit ammonia, rather than absorb it at high temperatures as explained in Introduction Atmosphere, 2021 (section 18.5). Vegetation is often not homogeneous, causing higher deposition rates at transitions from short to tall vegetation. Comparing model results with observed NH$_3$ deposition rates is an active field of research, where the level of agreement is an indication of how well the processes are understood. Actually, the ammonia budget of the Netherlands as we understand it, is not entirely closed. There is a gap between the observed and simulated concentrations which suggests that either the emissions are larger than we know it, or the deposition rates are lower. However, the process model you will develop through the following exercises may be considered a good first order description of the deposition process.

### 3.1 Aerodynamical resistance 

The aerodynamical resistance can be calculated as:

\begin{equation*}
r_a = \frac{\left( \ln \frac{z-z_d}{z_0} \right)^2}{k^2u},
\end{equation*}

where $z$ (m) is the height of the reference level where the wind speed is measured, $z_d$ (m) is the zero-plane displacement height, $z_0$ (m) is the surface roughness length, k (-) is the von Karman constant and $u$ (m s$^{-1}$) is the wind speed. Note that this equation implicitly assumes neutral conditions. In stably stratified or unstable conditions, $r_a$ will be larger or smaller, respectively, than expressed through the presented equation. The error that arises due to our assumption is small for rough surfaces, like forests (where the surface roughness length is equal 1), but larger for smooth surfaces (where the surface roughness length is smaller than 1). There are methods to account for this, but it would make the approach a bit more complex, thus, we will work for now with the presented equation.

### Exercise 3a: Aerodynamical resistance analysis

In the following exercise, you will analyze the aerodynamical resistance for the land use type at the study area of your PBL group. First, let us load the necessary data collected at the Veenkampen meteorological station, and then you will calculate the aerodynamical resistance. 

In [ ]:
# Load the meteorological data. 
# Load the velocity (u; m/s), global radiation (Rg; W/m^2), rain (Rain; mm), rain in the last three hours 
# (Rain_last3h; mm) and leaf area index (LAI; m^2/m^2).
df_meteo = pd.read_csv(datapath+'Data_Meteo_Veenkampen_2013-2021.csv', sep=';', 
    index_col='date-time', usecols=['date-time', 'Rg', 'u', 'P'], parse_dates=['date-time'])
df_meteo.columns = ['Rg', 'u', 'P']

### Question 3.1: 

1. How does the aerodynamical resistance act compared to the wind speed? <br>
2. When is the aerodynamical resistance higher, during the day or night, and why? <br>

To answer question 3.1 do the following:

<ul>
<li><p style="background:powderblue;color:black">Set the constants needed to calculate the aerodynamical resistance. Those are the von Karman constant $k = 0.4$, the height of the reference level where the wind speed is measured $z = 10$ m, the zero-plane displacement height $z_d = 0$ m, and the surface roughness length $z_0$ [m]. </li></ul>

- Find the values for the surface roughness length in Table 1 of the paper by [Kumar et al. (2011)](https://link.springer.com/article/10.1007%2Fs10546-010-9559-z). Note that for the the surface roughness length $z_0$ you should use a mean between the minimum and maximum roughness length.

<ul>
<li><p style="background:powderblue;color:black">Calculate the aerodynamical resistance.</li><br>
<li><p style="background:powderblue;color:black">Add the wind speed and the aerodynamical resistance to the same matrix for easier plotting.</li><br>
<li><p style="background:powderblue;color:black">Plot the result on the screen. We plot the time series of the wind speed [m/s] on the top figure and the aerodynamical resistance [s/m] on the bottom figure.</li></ul>

- Analyse the wind and the resistance, and answer the posed questions in the second box below. To answer your questions you will fill in the following sentences: <br>
`The aerodynamical resistance (x = 'increases/decreases') with increasing wind speed.` <br>
`The aerodynamical resistance is slightly higher during the (y = 'day/night').`<br>
`This happens because of (z = 'stronger/weaker') vertical mixing.`

NB: You can zoom in in the figure. The other panel will automatically adjust to the new scale too.

In [ ]:
# Set the constants needed for calculation of the aerodynamical resistance.  
k  = 0.4        # - Karman constant
z  = 10.        # m Reference height (height of measurements)
zd = 0          # m Zero-plane displacement height 
z0 = 0.11       # m Get the number from the table for your land type

# Calculate the aerodynamical resistance based on the formula given in section 3.1.
ra = (np.log((z-zd)/z0))**2/(k**2*df_meteo['u'])

# Plot the wind speed (u, top figure) and the aerodynamical resistance (ra, bottom figure).
# Plot Time [hours] on x-axis and wind speed [m/s], or Resistance [s/m] on y-axis. 
df_meteo_plot = pd.concat([df_meteo['u'], ra], axis=1, sort=False)
df_meteo_plot.columns = ['u', 'ra']

fig6 = df_meteo_plot[['u', 'ra']].iplot(asFigure=True, subplots=True, shape=(2,1), shared_xaxes=True,  
    layout=dict(yaxis=dict(title='Wind speed [m s\u207B\u00B9]'), xaxis=dict(title=' ')))
fig6['layout']['yaxis2'].update({'title':'Resistance [s m\u207B\u00B9]'})
fig6['layout']['xaxis2'].update({'title':'Time [years]'})
fig6.show()

del(df_meteo_plot)

In [ ]:
# Write your answers. N.B.: You can zoom into the figure to find the answers.
# x: The aerodynamical resistance (x = 'increases/decreases') with increasing wind speed.
# y: The aerodynamical resistance is slightly higher during the (y = 'day/night').
# z: This happens because of (z = 'stronger/weaker') vertical mixing.
x = 'decreases'    
y = 'night'    
z = 'weaker'    

ss.check_my_answer_AQ3_1(x, y, z)

### 3.2 Canopy resistance

Most vegetation types change the opening of their stomates to regulate how much CO$_2$ they take from the atmosphere and how much water vapor they lose. When the stomates are open, other species, such as NH$_3$ and O$_3$, can also be exchanged. When referring to the resistance at the leaf level, we use the term stomatal resistance. For tall vegetation structures, the stomatal resistance varies between the top and lower vegetation levels. Thus, when referring to the resistance at the vegetation level we use the term canopy resistance $r_c$ [s m$^{-1}$]. The canopy resistance accounts for those differences (i.e., the differences between the top and lower vegetation levels).

There are two main approaches to model the canopy resistance. The first is the Jarvis approach. [Paul Jarvis](https://en.wikipedia.org/wiki/Paul_Gordon_Jarvis) (1935-2013) studied how canopy resistance responds to atmospheric conditions (e.g., vapor pressure deficit, radiation and temperature) and soil conditions (e.g., soil moisture and nutrient availability). He came up with a model formulation which assumes that each vegetation type has an intrinsic minimum resistance when the stomates are fully opened in optimal conditions. This minimum resistance is increased when the vegetation experiences stress, e.g., when the air is dry and the vegetation needs to reduce water loss. [Jarvis (1976)’s](https://royalsocietypublishing.org/doi/10.1098/rstb.1976.0035) formulation assumes that the vegetation response to each stress factor can be described by an independent multiplicative term. This makes the Jarvis approach simple to use and it also facilitates comparison between vegetation types.

The second is the A-g$_s$ approach, where A stands for CO$_2$ assimilation and $g_s$ for canopy conductance, i.e., the inverse of canopy resistance (see e.g., [Jacobs and de Bruin, 1997](https://journals.ametsoc.org/doi/full/10.1175/1520-0450%281997%29036%3C1663%3APRTAEA%3E2.0.CO%3B2) and [Ronda et al., 2001](https://journals.ametsoc.org/doi/full/10.1175/1520-0450%282001%29040%3C1431%3AROTCCI%3E2.0.CO%3B2)). This approach is based on newer scientific evidence that vegetation optimizes the stomatal resistance for the specific carbon uptake for minimal water loss. Some vegetation types respond rather different to environmental stress factors than others. For example, some species use water rather aggressively to grow faster, but at the risk of running out of water. Other species use water rather conservatively to prevent running out of water, but at the cost of growing slower (e.g., [van der Molen et al., 2010](https://www.sciencedirect.com/science/article/pii/S0168192311000517?via%3Dihub)). Taking into account the water cost of carbon uptake requires continuous integration between water availability, water loss and carbon uptake capacity of the vegetation. 

The Jarvis approach describes the canopy resistance as a minimum canopy resistance $r_{c,min}$ [s m$^{-1}$], which applies to optimal conditions. In stressed conditions, the actual canopy resistance $r_c$ [s m$^{-1}$] is larger than the minimum, depending on the stress terms $F_1$, $F_2$, ..., $F_n$:

\begin{equation*}
r_c = \frac{r_{c,min}}{LAI F_1F_2...F_n}.
\end{equation*}

The $LAI$ [m$^2$ m$^{-2}$] in the denominator is leaf-area index which implies that at each vegetation level $i$  $r_c$ decreases linearly. This is arguably a simplistic approach, because lower levels may be shaded and cooler than higher layers. The stress functions $F_1$, ..., $F_n$ quantify the level of stress due to environmental processes. The stress functions are given as:

\begin{equation*}
F_1 = \frac{\frac{r_{c,min}}{r_{c,max}}+f}{1+f}, \\
F_2 = \frac{1}{1+h_s[q_s(T_a)-q_a]}, \\
F_3 = 1-0.0016(T_{ref}-T_a)^2, \\
F_4 = \sum_{i=1}^{n_{root}}\frac{(\theta_i-\theta_{wilt})d_i}{(\theta_{ref}-\theta_{wilt})d_{tot}}.
\end{equation*}


Here $r_{c,min}$ and $r_{c,max}$ [s m$^{-1}$] are minimum and maximum canopy resistances, respectively. We will initially work only with $F_1$ for simplicity, while neglecting $F_2$ to $F_4$. In the equation for $F_1$, $f=0.55\frac{R_g}{R_{gl}}\frac{2}{LAI}$, where $R_g$ [W/m$^2$] is the global radiation, $R_{gl}$ [W m$^{-2}$] is the minimum solar radiation necessary for photosynthesis (transpiration) to occur. 

Just to be complete, we give the descriptions of $F_2$ to $F_4$ as well. In the equation for $F_2$, $h_s$ [-] is a parameter associated with the water vapour deficit, $T_a$ [K] is air temperature at reference level, $q_s$ and $q_a$ [g kg$^{-1}$ or kg kg$^{-1}$] are saturation and actual water vapor mixing ratios, respectively. They can be calculated as:

\begin{equation*}
q_s = \frac{R_d}{R_v}\frac{0.6107\cdot10^{\frac{7.5T_a}{237.3+T_a}}}{p}, \\
q_a = RH\cdot q_s,
\end{equation*}

where $R_d = 287$ J kg$^{-1}$ K$^{-1}$ and $R_v = 462$ J kg$^{-1}$ K$^{-1}$ are the gass constants for dry air and water vapour, respectively, $p \approx 1.1$ kPa is the air pressure at the sea level, and $RH$ [-] is the relative humidity. These parameters should be chosen in such a way that $F_2$ is a number between 1 and ~10. In the equation for $F_3$, $T_{ref}$ [K] is the optimal air temperature for photosynthesis, while $T_a$ [K] is the air temperature. Finally, in the equation for $F_4$, $\theta$ [m$^3$ m$^{-3}$] is volumetric water content, while and $d_i$ and $d_{tot}$ [m] are the depth of soil layer i and the total root depth. This formulation and a more detailed description can be found in [Kumar et al. (2011)](https://link.springer.com/article/10.1007%2Fs10546-010-9559-z). Now, let's use this approach to simulate the canopy resistance $r_c$.

### Exercise 3b: Canopy resistance analysis

In this exercise, you will calculate and analyse the cannopy resistance based on your chosen land surface type. As we saw before, cannopy resistance depends on minimum canopy resistance $r_{c,min}$, leaf area index $LAI$ and the stress factors $F_1,...,F_4$. In practice, $F_1$ is the stress factor for solar radiation which describes a large part of the diurnal variability in the canopy resistance $r_c$. To start simply and straightforward, we can (for now) assume that $F_2 = F_3 = F_4 = 1$, implying that the vegetation does not experience stress because of dry air, high or low temperatures and dry soil. 

### Question 3.2: 

1. Does the value of the canopy resistance fall in between the minimum and the maximum canopy resistance? <br>
2. What can you say about the resistance during summer and winter? <br>
3. How is the global radiation related to the canopy resistance? <br>
4. How is stomata connected to the canopy resistance? <br>

To answer question 3.2 do the following:

<ul>
<li><p style="background:powderblue;color:black">Set the constants needed to calculate the canopy resistance. Those are the the minimum canopy resistance  rcmin [s m$^{-1}$], the sensitivity to global radiation Rgl [W m$^{-2}$], the leaf area index LAI [m$^2$ s$^{-2}$] and the maximum canopy resistance rcmax [s m$^{-1}$]. You can find the values for rcmin, Rgl and LAI in Table 1 of the paper by Kumar et al. (2011). Note that it is better to simulate a vegetated land surface type at this stage, because referring to water bodies is too simple. The value of rcmax represents the canopy resistance when the stomates are closed. Exchange between the atmosphere and the vegetation is much slower then, because gasses must diffuse through the epidernis (i.e., leaf skin). The exact value of rcmax is not very sensitive, as long as it is large, therefore a value of 1000-10000 s m$^{-1}$ will suffice. </li><br>
<li><p style="background:powderblue;color:black">Calculate f (in F1) and the stress function for solar radiation F1. </li><br>
<li><p style="background:powderblue;color:black">Calculate the canopy resistance rc. </li><br>
<li><p style="background:powderblue;color:black">Plot the result on the screen. We plot the time series of the canopy resistance [s m$^{-1}$].</li></ul>

- Analyse the wind and the resistance, and answer the posed questions in the second cell box below. To answer your questions you will fill in the following sentences:<br>
`The value of the canopy resistance (x = 'does/does not') fall in between the minimum and the maximum canopy resistance.` <br> 
`The resistance is (y = 'higher/lower') during the summer.` <br>
`The global radiation is (z = 'proportional/inversely proportional') to the canopy resistance.` <br>
`Stomata will (u1 = 'open up/close') with high radiation which leads to a (u2 = 'higher/lower') resistance.`

In [ ]:
# Set the constants needed to calculate the canopy resistance.
rcmin = 70.     # s/m   Get the number for your land type from the table
Rgl   = 65.     # W/m2  Get the number for your land type from the table
LAI   = 4.      # m2/m2 Get the number for your land type from the table
rcmax = 10000.  # s/m   Exact value does not matter so much as long as it is large.

# Calculate f and the stress function for solar radiation F1.
f     = 0.55*(df_meteo['Rg']/Rgl)*(2./LAI)
F1    = ((rcmin/rcmax)+f)/(1+f)
F2    = 1.0          # We do not work with F2...F4. You can do so later in BasicSkills_AirQuality_part2.ipynb
F3    = 1.0
F4    = 1.0 

# Calculate the canopy resistance in s/m based on the formula given in the markdown-box above.
rc    = rcmin/(LAI*F1*F2*F3*F4)  

# Plot the canopy resistance.
# Plot Time [years] on x-axis and Resistance [s/m] on y-axis.
fig8 = rc.iplot(asFigure=True, xTitle='Time [years]', yTitle='Resistance [s m\u207B\u00B9]', width=2)
fig8['layout']['yaxis'].update({'range': [rc.min(), rc.min()+20.]}) 
fig8.show()

In [ ]:
# Write your answers. N.B.: Look careful at this figure. The orange lines represent the data, the gray areas indicate 
# absence of data. Zoom in to get a better impression!

# x: 'The actual value of the canopy resistance rc (x = 'is/is not') limited to in between the prescribed values of 
#     the minimum and the maximum canopy resistances (rcmin and rcmax).' (... Why (not)?...)
# y: 'The resistance is (y = 'higher/lower') during the summer.'
# z: 'The global radiation is (z = 'proportional/inversely proportional') to the canopy resistance.'
# u1/u2: `Stomates will (u1 = 'open up/close') with high radiation which leads to 
# a (u2 = 'higher/lower') resistance.'
x  = 'is not'
y  = 'lower'
z  = 'inversely proportional'
u1 = 'open up'
u2 = 'lower'

ss.check_my_answer_AQ4_1(x, y, z, u1, u2)

### Section 3.3: Sub-laminar boundary layer resistance

A thin layer (a few millimeters) of laminar air forms close to the leaf surface. Transport in laminar flows is an order of ~100 slower than turbulent transport. This adds a small but rather constant sub-laminar boundary layer resistance $r_b$ [s m$^{-1}$] against exchange between atmosphere and vegetation. The value of $r_b$ depends on the size, shape and roughness of the leaf surface. Some leafs have a hairy surface which increases the resistance. In practice, the exact value of $r_b$ is usually small relative to the aerodynamic and canopy resistances, except when the atmosphere is very turbulent and the stomates are fully open. In those situations $r_b$ makes up small but significant fraction of the total resistance. A typical value is $r_b = 5$ s m$^{-1}$.

### Exercise 3.3: Resistances comparison

In this exercise, you will compare the three resistances.

### Question 3.3:

1. What are the typical values of of $r_a$ and $r_c$ in winter months and how do they compare to $r_b$? <br>
2. Which resistance has the highest diurnal variability? <br>
3. Can you explan the change in maxima and minima of this resistance during the day? <br>

To answer question 3.3 look at the figures above and fill in the following sentences:

`In winter months aerodynamic_res = number and canopy_res = number. Boundary layer resistance $r_b$ is (x = 'higher/lower/negligible') compared to the aerodynamical and canopy resistance.` <br>
`In the figures above can be seen that (y1 = 'aerodynamical/canopy/boundary layer') resistance has the highest diurnal variability. It depends on (y2 = 'radiation/roughness length').` <br>
`During the (z1 = 'day/night') there is no photosynthesis which leads to (z2 = 'high/low') canopy resistance.`

In [ ]:
# Write your answers.
# aerodynamic_res: (integer) What are typical values of ra in winter months (s/m)? Choose from 20, 100, 500
# canopy_res: (integer) What are typical minimal (midday) values of rc in winter months (s/m)? Choose from 15, 30, 50, 500
# x: rb is (x = 'higher/lower/negligible') compared to the typical ra and rc.
# y1: The (y1 = 'aerodynamical/canopy/boundary layer') resistance has the highest diurnal variability.
# y2: This resistance depends on (y2 = 'radiation/roughness length').
# z1, z2: During the (z1 = 'day/night') there is no photosynthesis 
# which leads to (z2 = 'high/low') canopy resistance.
aerodynamic_res = 20  
canopy_res      = 30 
x  = 'negligible'                
y1 = 'canopy'                
y2 = 'radiation'                
z1 = 'night'                
z2 = 'high'                

ss.check_my_answer_AQ5_1(aerodynamic_res, canopy_res, x, y1, y2, z1, z2)

### Exercise 3.4: Total resistance and dry deposition flux

In this exercise, you will calculate the total resistance and finalize the calculation of the dry deposition flux for the three stations i.e., Wekerom, Vredepeel and Zegveld. 

### Question 3.4:

1. Which station has the highest deposition flux? <br>
2. What leads you to that conclusion? <br>
3. How does the deposition change during the year? <br>

To answer question 6.1 do the following:

<ul>
<li><p style="background:powderblue;color:black"> Calculate the total resistance as a sum of the aerodynamic $r_a$ [s m$^{-1}$], boundary layer $r_b$ [s m$^{-1}$] and canopy resistance $r_c$ [s m$^{-1}$]. This you can do as following: `rt = ra + rb + rc`. But first let's not forget to set the boundary layer resistance to 5 s m$^{-1}$, i.e., `rb = 5`.</li><br>
<li><p style="background:powderblue;color:black"> Calculate the dry deposition flux $F_{NH_3}$ based on $F_{NH_3} = \frac{C_{NH_3,\alpha}-C_{NH_{3,0}}}{r_t}$. The NH$_3$ concentration at the zero level $C_{NH_{3,0}}$ may be assumed to be 0 $μg m^{-3}$, because deposition in the water film on the interior of the stomates is so efficient that it removes all NH$_3$ from the air in its immediate surrounding. You will calculate the dry deposition flux for each station separately as following: `F_NH3_stationname = df_airquality['NH3_stationname']/rt`, where `stationname` corresponds to `Wekerom`, `Vredepeel` and `Zegveld`.</li><br>
<li><p style="background:powderblue;color:black">Make a matrix with deposition rates of all stations for easier plotting.</li><br>
<li><p style="background:powderblue;color:black">Plot the result on the screen.  Here we plot a time series of the deposition flux in μg m$^{-2}$ s$^{-1}$ for the three stations (Wekerom in orange, Vredepeel in blue, and Zegveld in green).</li></ul>
    
- Analyse the deposition flux and answer your questions. To answer the *first* question, you will write: `x = 'stationname'`. Possible answers to the first question are: x = 'Wekerom', 'Vredepeel', 'Zegveld'. To answer the *second* question, present a part of python code that gives you a number of how high deposition flux at that station is. To answer the *third* question you will  fill in the given sentence. <br>
`It seems that the deposition flux is (z1 = higher/lower) during summer compared to winter when it is (z2 = higher/lower).`

In [ ]:
# Calculate the total resistance as a sum of the aerodynamic (ra; s/m), boundary layer (rb, s/m) 
# and canopy resistance (rc; s/m).
rb = 5.
rt = ra + rb + rc

# Calculate the dry deposition rate (F_NH3; ug/m^2/s) using a gradient-resistance model 
# for all three stations, i.e., for Wekerom, Vredepeel and Zegveld.
F_NH3_Wekerom   = df_airquality['NH3_Wekerom'  ]/rt
F_NH3_Vredepeel = df_airquality['NH3_Vredepeel']/rt
F_NH3_Zegveld   = df_airquality['NH3_Zegveld'  ]/rt

# Add the matrix for plotting
df_meteo_plot = pd.concat([F_NH3_Wekerom, F_NH3_Vredepeel, F_NH3_Zegveld], axis=1, sort=False)
df_meteo_plot.columns = ['Wekerom', 'Vredepeel', 'Zegveld']

In [ ]:
# Plot the deposition rate for the three stations
# (Wekerom in orange, Vredepeel in blue and Zegveld in green). 
# Plot Time [years] on x-axis and Deposition rate [um/m^2/s] on y-axis. 
fig9 = df_meteo_plot.iplot(asFigure=True, subplots=True, shape=(3,1), 
    shared_xaxes=True, fill=True, layout=dict(yaxis=dict(title=' '), xaxis=dict(title=' ')))
fig9['layout']['yaxis2'].update({'title':'Deposition flux [\u00b5g m\u207B\u00b2 s\u207B\u00B9]'})
fig9['layout']['xaxis2'].update({'title':' '})    
fig9['layout']['xaxis3'].update({'title':'Time [years]'})   
fig9['layout']['yaxis1'].update({'range': [0, 6]}) 
fig9['layout']['yaxis2'].update({'range': [0, 6]}) 
fig9['layout']['yaxis3'].update({'range': [0, 6]}) 
fig9.show()

del(df_meteo_plot)

In [ ]:
# Answer the questions.
# x:      Which station has the highest deposition flux? (x = 'Stationname' (Wekerom, Vredepeel or Zegveld))
# y1:     Part of python code that gives a mean value of the deposition flux for a particular station. (y1 = 'code') 
# z1, z2: It seems that the deposition flux is (y1 = higher/lower) during summer 
# compared to winter when it is (y2 = higher/lower).
x  = 'Vredepeel'  
y  = 'F_NH3_Vredepeel.mean()'   
z1 = 'higher'
z2 = 'lower'
  
ss.check_my_answer_AQ6_1(x, y, z1, z2)

## Exercise 3.5a:

In this exercise you will look at the mean seasonal and daily variabilities. Thus, we will make calculations and plots for mean seasonal variability and mean daily variability for june and december. We again look at all three stations. 

N.B.: the exercises are similar as the ones you did for NH$_3$ concentration, but now we look at the NH$_3$ deposition flux. The depositioni flux depends on the concentration and the meteorology. The meteorology may either dampen or amplify the variations in the concentrations. You need to understand this in order to use the model in the PBL part of the course.

### Question 3.5a:

1. How does the NH$_3$ deposition rate vary on a seasonal time scale? <br>
2. Can you see a connection between the deposition rate variability and NH$_3$ concentration variability?

To answer question 3.5a do the following:

<ul>
<li><p style="background:powderblue;color:black">Add the calculated aerodynamic, canopy and total resistance, and the calculated deposition fluxes to the original dataframe. The benefit of a dataframe over a stand alone variable is that it inheritely contains a date-time index and methods to quickly compute weekly, monthly and annual averages and display them in figures and tables. You can do that by writing: `df_meteo['variable'] = variable`, where `df_meteo` is the name of our datafarme containing the meteorological data we loaded at the beginning of section 3 and `variable` is a name of the variable you wish to save. </li><br>
<li><p style="background:powderblue;color:black">Calculate the mean seasonal variability. This calculation you already performed in exercise 2b, but there you looked at the ammonia concentration. Now, you have to make the same calculation for the deposition flux. Do you still remember how to do that? Here is a tip: `df_meteo_mean_seasonal = df_meteo[['variable_station1', 'variable_station2', 'variable_station3']].groupby(2*((df_airquality.index.week-1)//2 + 1)).mean()`.Note that `variable_station1` corresponds to your deposition flux at a particular station (e.g., `F_NH3_Wekerom`).</li><br>
<li><p style="background:powderblue;color:black">Plot the result (mean seasonal variability) on the screen. Here we plot the time series in weeks of the deposition flux in $\mu g m^{-2} s^{-1}$.</li></ul>
    
- Answer your questions. To answer your questions, you will fill in the following sentences: <br>
`In general, the highest deposition flux is in (x = 'winter/spring/summer/autumn').` <br>
`During this time concentrations are (y1 = 'high/low') and the stomata are (y2 = 'open/closed'), so these two processes strengthen each other. `

In [ ]:
# Add the calculated resistances and deposition fluxes into the dataframe with the meteorological data.
df_meteo['ra']              = ra
df_meteo['rc']              = rc
df_meteo['rt']              = rt
df_meteo['F_NH3_Wekerom']   = F_NH3_Wekerom
df_meteo['F_NH3_Vredepeel'] = F_NH3_Vredepeel
df_meteo['F_NH3_Zegveld']   = F_NH3_Zegveld

# Calculate the mean seasonal variability for all three stations. 
df_meteo_mean_seasonal = df_meteo[['F_NH3_Wekerom', 'F_NH3_Vredepeel', 'F_NH3_Zegveld']].groupby(2*((df_meteo.index.isocalendar().week-1)//2 + 1)).mean()

# Plot the result on the screen for the three stations 
# (Wekerom in orange, Vredepeel in blue and Zegveld in green). 
# Plot Time [weeks] on x-axis and Deposition flux [um/m^2/s] on y-axis.
fig10 = df_meteo_mean_seasonal.iplot(asFigure=True, xTitle="Time [weeks]", 
                                     yTitle="Deposition flux [\u00b5g m\u207B\u00b2 s\u207B\u00B9]", width=2)
fig10.show()

In [ ]:
# Answer your questions.
# x: `In general, the highest deposition flux is in (x = 'winter/spring/summer/autumn').`
# y: `During this time concentrations are (y1 = 'high/low') and the stomata are (y2 = 'open/closed'), 
# so these two processes strengthen each other. `
x  = 'summer'
y1 = 'high'
y2 = 'open'

ss.check_my_answer_AQ7a_1(x, y1, y2)

## Exercise 3.5b:

Next, let's calculate the mean diurnal variability.

### Question 3.5b:

1. How does the NH$_3$ deposition rate change diurnally? <br>
2. Can you see a connection between the diurnal variability in the deposition rate and the diurnal variability in NH$_3$ concentration? <br>
3. What do you think is the most important cause for variability in the deposition flux? <br>

To answer question 3.5b do the following:

<ul>
<li><p style="background:powderblue;color:black">Since we want to see the difference between daily variability in summer and in winter, we need to select the data that correspond to one month in summer (e.g., June) and one month in winter (e.g., December). We already did this in exercise 2c as: `df_meteo_mon = df_meteo[['variable_station1', 'variable_station2', 'variable_station3']].loc[(df_meteo.index.month==number)]`, where in stead of `mon` you can write `june` or `december`, while in stead of `number` you can write `6` or `12`. Note that variable_station1 corresponds to your deposition flux at a particular station (e.g., `F_NH3_Wekerom`). </li><br>
<li><p style="background:powderblue;color:black">Calculate the mean daily variability for the three stations. Again, we make the calculation for all three stations together. We can calculate the mean daily variability as: `df_meteo_mean_mon_daily = df_meteo_mon.groupby([df_meteo_mon.index.hour]).mean()`. </li><br>
<li><p style="background:powderblue;color:black">Setup a new dataframe. This is important for easier plotting. We do this in two steps. In the first step we will concatinate the two dataframes (the one where we calculated the daily variability for june, and the one where we calculate the daily variability for december). This we can do as: `df_meteo_mean_mon1_mon2 = pd.concat([df_meteo_mean_mon1_daily, df_meteo_mean_mon2_daily], axis=1, sort=False)`. In the second step we setup the names of the newly defined variables. This can be done as: `df_meteo_mean_mon1_mon2.columns = ['F_NH3_station1_mon1', 'F_NH3_station2_mon1', 'F_NH3_station3_mon1', 'F_NH3_station1_mon2', 'F_NH3_station2_mon2', 'F_NH3_station3_mon2']`. Here, `station1`, `station2`, and `station3` correspond to `Wekerom`, `Vredepeel` and `Zegveld`, while `mon1` and `mon2` correspond to `june` and `december`, respectively.</li><br>
<li><p style="background:powderblue;color:black">Plot the mean daily variability for all three stations. In this figure, we plot the Time [hours] on x-axis and the NH$_3$ Deposition [um/m$^2$/s] on y-axis.</li></ul>
    
- In the second cell box below, analyse the daily variability and fill in the sentences. <br>
`The deposition flux is (x = 'higher/lower') during the day.` <br>
`Ammonia concentrations are higher during the (y = 'day/night') which should lead to higher deposition flux.` <br>
`The deposition flux is zero during the (z1 = 'day/night') because (z2 = 'aerdynamic resistance/canopy resistance') becomes infinitely large.`</ul></ol>

In [ ]:
# Extract June and December from the deposition rate time series.
df_meteo_jun = df_meteo[['F_NH3_Wekerom', 'F_NH3_Vredepeel', 'F_NH3_Zegveld']].loc[(df_meteo.index.month==6 )]
df_meteo_dec = df_meteo[['F_NH3_Wekerom', 'F_NH3_Vredepeel', 'F_NH3_Zegveld']].loc[(df_meteo.index.month==12)]

# Calculate the diurnal variability in the deposition rate.
df_meteo_mean_jun_daily = df_meteo_jun.groupby([df_meteo_jun.index.hour]).mean()
df_meteo_mean_dec_daily = df_meteo_dec.groupby([df_meteo_dec.index.hour]).mean()

# We set up the new data (i.e., the daily variabilities) in a new matrix.
df_meteo_mean_jun_dec = pd.concat([df_meteo_mean_jun_daily, df_meteo_mean_dec_daily], axis=1, sort=False)
df_meteo_mean_jun_dec.columns = ['F_NH3_Wekerom_J', 'F_NH3_Vredepeel_J', 'F_NH3_Zegveld_J', 'F_NH3_Wekerom_D', 'F_NH3_Vredepeel_D', 'F_NH3_Zegveld_D']

# Plot the result (mean diurnal variability for June and December) for all three stations 
# (Wekerom in orange, Vredepeel in blue and Zegveld in green). 
# Results for June are given with solid lines and results for December with dotted lines. 
# Plot Time [hours] on x-axis and Dry deposition rate [um/m^2/s] on y-axis. 
fig11 = df_meteo_mean_jun_dec.iplot(asFigure=True, xTitle="Time [hours]", 
                                    yTitle="Deposition flux [\u00b5g m\u207B\u00b2 s\u207B\u00B9]", 
                                    colors=['orange', 'blue', 'green', 'orange', 'blue', 'green'], 
                                    dash=['solid', 'solid', 'solid', 'dot', 'dot', 'dot'], width=2,tickvals=[0,3,6,9,12,15,18,21,24])
fig11.show()
#fig11.get_xticks()

In [ ]:
# Answer your questions. 
# x: The deposition flux is (x = 'higher/lower') during the day.
# y: Ammonia concentrations are higher during the (y = 'day/night') which should lead to higher depoisition flux.
# z: The deposition flux is zero during the (z1 = 'day/night') because (z2 = 'aerdynamic resistance/canopy resistance') 
#    becomes infinitely large.
x  = 'higher'
y  = 'night' 
z1 = 'night'
z2 = 'canopy resistance'

ss.check_my_answer_AQ7b_1(x, y, z1, z2)

### Question 3.5c: 

1. How high is the deposition flux at each station in 2018? <br>
2. Can you compare the deposition fluxes you calculated for the three stations with the fluxes shown in Figure 1? <br>

To answer question 3.5c do the following:

<ul>
<li><p style="background:powderblue;color:black">Calculate the inter-annual mean deposition flux for all stations. You can do that as: `mean_deposition_flux = df_meteo[['F_NH3_station1', 'F_NH3_station2', 'F_NH3_station3']].resample('Y').mean()`, where `station1` corresponds to `Wekerom` and so on. </li></ul>

- Convert μg/m2/s to mole/ha/yr. To do so do the following: <br>
  1. convert μg to g using the constant ug_to_g. Note that 1 g is 10$^6$  μg.
  1. convert g to mole using the constant g_to_mole. Note that 1 mole NH$_3$ weighs 17 g. 
  1. convert m$^{-2}$ to ha$^{-1}$ using the constant m2_to_ha (this is reversed because of the inverse position). Note that 1 ha equals 10$^4$ m$^2$.
  1. convert s$^{-1}$ to yr$^{-1}$ using the constant second_to_year (this is reversed because of the inverse position). <br>
  
- Write your result as mole/ha/yr. To do so, you have to multiply your annual deposition flux with the conversion constants you just calculated. Thus, your result should be calculated as: `mean_deposition_flux_new = mean_deposition_flux*conversion_variable1*conversion_variable2*conversion_variable3*conversion_variable4`.<br>

- Write the result on your screen. <br>

- Answer your questions in the second cell box below. To answer your questions fill in the sentences: <br>
`The deposition flux obtained here are: stationname = number.`<br>
`The deposition fluxes given in Figure 1 are (x = 'comparable/not comparable') to the ones calculated here.` <br></ul></ol>

In [ ]:
# Fill in the variables you want to output.
mean_deposition_flux = df_meteo[['F_NH3_Wekerom', 'F_NH3_Vredepeel', 'F_NH3_Zegveld']].resample('Y').mean()

# Convert ug/m2/s to mole/ha/yr: 
# (Make sure to make the conversion factors floating points by adding a decimal point, 
# to avoid integer division becoming a modulo division, e.g: 17 --> 17.)
ug_to_g        = 0.000001             # 1 ug     equals ... g
g_to_mole      = 1.0/17.0             # 1 g NH3  equals ... mole
m2_to_ha       = 1.0/10000.           # 1 m2     equals ... ha
second_to_year = 1.0/(365*24*60*60)   # 1 second equals ... year
                            # hint: all conversion factors are < 1

# Calculate the deposition flux in mole/ha/yr.
mean_deposition_flux_new = mean_deposition_flux * ug_to_g * g_to_mole / m2_to_ha / second_to_year

# Print the result on screen.
mean_deposition_flux_new.index = mean_deposition_flux_new.index.strftime("%Y")
pd.options.display.float_format = '{:,.0f}'.format
print('Deposition flux in mole/ha/yr is:')
print(mean_deposition_flux_new)

In [ ]:
# Write your answers.
# stationname: `The deposition flux obtained here are: stationname = number.`
# x: `The deposition fluxes given in Figure 1 are (x = 'comparable/not comparable') to the ones calculated here.`
F_NH3_2018_Wekerom   = 1884          # mole/ha/yr
F_NH3_2018_Vredepeel = 2813          # mole/ha/yr
F_NH3_2018_Zegveld   = 1171          # mole/ha/yr
x                    = 'comparable'

ss.check_my_answer_AQ7b_2(F_NH3_2018_Wekerom, F_NH3_2018_Vredepeel, F_NH3_2018_Zegveld, x)

### 4 SAVE YOUR RESULTS, CLEAR OUTPUT and CLOSE NOTEBOOK
You reached the end of this first part of your practical entitled Basic Skills Air Quality. Here at the end, let's save our data (meteorological data that we initially loaded and resistances and deposition rate that we here calculated) into a new csv file. You can always change the name of this file, but note that in the next notebook you need to load it under the same name that you save it under.

In [ ]:
df_meteo.to_csv('./Data_Output_BasicSkills_AirQuality.csv', sep=';')

Now, to limit the data storage please do the following:
- go to *KERNEL* --> *RESTART and CLEAR OUTPUT*. This removes the interactive graphs from your notebook, but leaves your answers intact.
- go to *FILE* --> *SAVE and CHECKPOINT*. This saves your answers in the notebook
- go to *FILE* --> *CLOSE and HALT*, to shutdown the notebook. You can always come back later and restart.